In [32]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from omegaconf import OmegaConf
import mlflow

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

import tensorflow as tf

Dense = tf.keras.layers.Dense
Input = tf.keras.layers.Input
Sequential = tf.keras.Sequential
MeanSquaredError = tf.keras.losses.MeanSquaredError
BinaryCrossentropy = tf.keras.losses.BinaryCrossentropy
Sigmoid = tf.keras.activations.sigmoid

file_path = os.getcwd()

conf = OmegaConf.load(os.path.join(file_path, "..", "src", "config.yml"))

mlflow.set_experiment(conf["tracking_uri"]["experiment_name"])

data_path = os.path.join(file_path, "..", "data", "UCI_Credit_Card.csv")

df = pd.read_csv(data_path)

lambdas = [0, 1e-5, 1e-4, 5e-4, 1e-3]

In [33]:
def prepare_data(df, params):

    # ===============================
    # TARGET
    # ===============================
    y = df['default.payment.next.month']

    # ===============================
    # FEATURES
    # ===============================
    X = df[
        [
            'LIMIT_BAL',
            'SEX',
            'EDUCATION',
            'MARRIAGE',
            'AGE',
            'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
            'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
            'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
            'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
            'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
        ]
    ]
    
    # ===============================
    # DEFINIÇÃO DOS TIPOS
    # ===============================
    binary_features = ['SEX']
    categorical_features = ['EDUCATION', 'MARRIAGE']
    numerical_features = [
        'LIMIT_BAL',
        'AGE',
        'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
        'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
        'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
        'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
        'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
    ]
    
    # ===============================
    # PREPROCESSAMENTO
    # ===============================
    preprocessor = ColumnTransformer(
        transformers=[
            ('bin', MinMaxScaler(), binary_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
            ('num', MinMaxScaler(), numerical_features)
        ]
    )
    
    # ===============================
    # SPLIT
    # ===============================
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=params["test_size"],
        random_state=params["random_state"],
        stratify=y
    )
    
	# ===============================
    # TRANSFORMAR DADOS
    # ===============================
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test

In [34]:
def cross_validate_lambda(X, y, lambdas, params):

    skf = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=params["random_state"]
    )

    cv_results = []

    for lambda_ in lambdas:
        aucs = []

        for train_idx, val_idx in skf.split(X, y):

            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = Sequential([
                Dense(64, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
                Dense(32, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
				Dense(1, activation = 'linear', name = 'layer3')
			], name = 'Cross_Default_Model')

            model.compile(
                optimizer=tf.keras.optimizers.Adam(
                    learning_rate=params["learning_rate"]
                ),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
            )

            model.fit(
                X_tr,
                y_tr,
                epochs=params["epochs_cross"],
                verbose=params["verbose"]
            )

            logits = model(X_val)
            y_pred_proba = tf.nn.sigmoid(logits).numpy().ravel()

            aucs.append(roc_auc_score(y_val, y_pred_proba))

        cv_results.append({
            "lambda": lambda_,
            "mean_auc": np.mean(aucs),
            "std_auc": np.std(aucs)
        })

        print(
            f"[CV] λ={lambda_:.3f} | "
            f"AUC={np.mean(aucs):.4f} ± {np.std(aucs):.4f}"
        )

    best_lambda = max(cv_results, key=lambda x: x["mean_auc"])["lambda"]

    return best_lambda

In [35]:
def train_final_model(X_train, X_test, y_train, y_test, best_lambda, params):

    model = Sequential([
        Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
        Dense(32, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
       	Dense(1, activation = 'linear', name = 'layer3')
	], name = 'Credit_Default_Model')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=params["learning_rate"]
        ),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    )

    model.fit(
        X_train,
        y_train,
        epochs=params["epochs"],
        verbose=params["verbose"]
    )

    logits = model(X_test)
    y_pred_proba = tf.nn.sigmoid(logits).numpy().ravel()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    print("\n📊 TEST SET FINAL")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

    return model


In [36]:
def main():
     
    # Preparação dos dados
    X_train, X_test, y_train, y_test = prepare_data(df, conf["parameters"])

    best_lambda = cross_validate_lambda(X_train, y_train, lambdas, conf["parameters"])

    print(f"\n🏆 Melhor lambda escolhido via CV: {best_lambda}")

    train_final_model(X_train, X_test, y_train, y_test, best_lambda, conf["parameters"])

if __name__ == "__main__":
    main()


[CV] λ=0.000 | AUC=0.7611 ± 0.0129
[CV] λ=0.000 | AUC=0.7618 ± 0.0074
[CV] λ=0.000 | AUC=0.7544 ± 0.0030
[CV] λ=0.001 | AUC=0.7455 ± 0.0076
[CV] λ=0.001 | AUC=0.7384 ± 0.0051

🏆 Melhor lambda escolhido via CV: 1e-05

📊 TEST SET FINAL
Accuracy: 0.8186
ROC-AUC: 0.7735
